# Dissertation

## Research into the techniques and methods to achieve state of the art accuracy in flower species identification


In [1]:
###imports###
import numpy as np
import tensorflow as tf
import scipy.io
from scipy import io
import time
import sys
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing import image
from tensorflow.python.framework import graph_util
from tensorflow.keras.callbacks import TensorBoard
import pickle
import datetime
import time
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import OneHotEncoder
import logging
#import skimage.io
import random
import PIL
from pathlib import Path
from keras.models import Model

Using TensorFlow backend.


In [2]:
print(tf.__version__)

2.3.0


In [3]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [4]:
# if tf.test.gpu_device_name(): 

#     print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

# else:

#     print("Please install GPU version of TF")

In [5]:
#makes all outputs be in float format rather than exponentials
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [7]:
#Set Parameters
IMG_SIZE = 224 #this parameter sets image dimensions as 50*50
DATE = datetime.datetime.now().strftime('%d-%b-%Y')
MODEL_PATH = f'models/{DATE}/'
MODEL_NAME = 'TransferResNet152V2.model'.format(int(time.time()))
TENSORBOARD = TensorBoard(log_dir=f'logs\\{MODEL_NAME}')

In [8]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Load Directory

In [9]:
URL_dir = 'E:/Github/thesis/flowerDataset/Flower'
labels_dir = 'E:/Github/thesis/flowerDataset/imagelabels.mat'
dataSplit_dir = 'E:/Github/thesis/flowerDataset/setid.mat'
loaded_images_dir = 'E:/Dissertation/loaded_images.npy'
image_train_dir ='E:/Dissertation/image_train.npy'
label_train_dir = 'E:/Dissertation/label_train.npy'
image_test_dir = 'E:/Dissertation/image_test.npy'
label_test_dir = 'E:/Dissertation/label_test.npy'
image_val_dir = 'E:/Dissertation/image_val.npy'
label_val_dir = 'E:/Dissertation/label_val.npy'

In [10]:
data_labels = io.loadmat(labels_dir)
data_labels = data_labels.items()
data_labels = list(data_labels)

In [11]:
df = pd.DataFrame(data_labels)

In [12]:
labels = df[1][3][0] #loads the labels that is stored in dataframe format

In [13]:
len(labels)

8189

In [14]:
#Got loading files code from link below
#https://stackoverflow.com/questions/30230592/loading-all-images-using-imread-from-a-given-folder
#Function gets all the images loaded up
def load_images_from_folder(URL):
    images = []
    for filename in os.listdir(URL):
        #img = cv2.imread(os.path.join(URL,filename))
        img = cv2.resize(cv2.imread(os.path.join(URL,filename), cv2.COLOR_BGR2RGB), (IMG_SIZE, IMG_SIZE))
        img = np.reshape(img,[IMG_SIZE,IMG_SIZE,3])
        if img is not None:
            images.append(img)
    np.save(loaded_images_dir, images)
#################################    return images#######################################################################

In [15]:
#No need to run this cell if image already loaded.
load_images_from_folder(URL_dir)

In [16]:
image_data = np.load(loaded_images_dir, allow_pickle=True)

In [17]:
image_data.shape

(8189, 224, 224, 3)

In [18]:
def randomised_dataset(image_data, labels):
    np.random.seed(8)
    indices = np.arange(image_data.shape[0])
    print(indices)
    np.random.shuffle(indices)
    image_data = image_data[indices]
    labels = labels[indices]
    return image_data, labels

In [19]:
#image_data, labels = randomised_dataset(image_data, labels)

In [20]:
# # Label Encoder

mlb = LabelBinarizer()
converted_labels = np.array(mlb.fit_transform(labels))

In [21]:
converted_labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [22]:
#citation code from https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
#splitting the image dataset into the ratio for training, validation and testing data
def split_data(image_data, converted_labels):
    
    train_ratio = 0.75
    validation_ratio = 0.15
    test_ratio = 0.10

    image_train, image_test, label_train, label_test = train_test_split(image_data, converted_labels, test_size=1 - train_ratio, random_state=42)

    #the test from previous line which is 25% of dataset is passed into the line below to be 
    #further split into 15% for validation and 10% for testing

    image_val, image_test, label_val, label_test = train_test_split(image_test, label_test, test_size=test_ratio/(test_ratio + validation_ratio),random_state=42) 

    print('image_train',image_train.shape)
    print('label_train',label_train.shape)
    print('image_test',image_test.shape)
    print('label_test',label_test.shape)
    print('image_val', image_val.shape)
    print('label_val', label_val.shape)

    np.save(image_train_dir, image_train)
    np.save(label_train_dir, label_train)
    np.save(image_test_dir, image_test)
    np.save(label_test_dir, label_test)
    np.save(image_val_dir, image_val)
    np.save(label_val_dir, label_val)



In [23]:
split_data(image_data,converted_labels)

image_train (6141, 224, 224, 3)
label_train (6141, 102)
image_test (820, 224, 224, 3)
label_test (820, 102)
image_val (1228, 224, 224, 3)
label_val (1228, 102)


In [24]:
image_train = np.load(image_train_dir, allow_pickle=True)
image_test = np.load(image_test_dir, allow_pickle=True)
label_train = np.load(label_train_dir, allow_pickle=True)
label_test = np.load(label_test_dir, allow_pickle=True)
image_val = np.load(image_val_dir, allow_pickle=True)
label_val = np.load(label_val_dir, allow_pickle=True)

In [26]:
image_train.shape

(6141, 224, 224, 3)

In [33]:
means = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))

In [34]:
stdevs = np.array([58.393, 57.12, 57.375]).reshape((1,1,1,3))

In [37]:
image_train = (image_train - means)/stdevs

In [38]:
image_test =(image_test - means)/stdevs

In [39]:
image_val =(image_val - means)/stdevs

### Feature Scaling

In [ ]:
# #converts the images to the range of 0 - 1.0 
# image_train = image_train/255.0
# image_test = image_test/255.0
# image_val = image_val/255.0

In [40]:
image_train.shape

(6141, 224, 224, 3)

In [41]:
label_train.shape

(6141, 102)

In [42]:
label_test.shape

(820, 102)

In [43]:
len(image_test)

820

## Find unique labels

In [44]:
len(label_train)

6141

In [45]:
# number of classes/ flowers
NUM_CLASSES = 102

In [46]:
label_train[:5]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0,

# Transfer Learning

In [47]:
from keras.applications import ResNet152V2

In [48]:
resnet = ResNet152V2(
    input_shape=(224,224,3),
    weights="imagenet",
    include_top=False, # removes last layer of the resnet model
)

In [49]:
for layer in resnet.layers: #don't train existing weights
    layer.trainable = False

In [50]:
x = Flatten()(resnet.output)

prediction = Dense(NUM_CLASSES, activation="softmax")(x)

resnet_model = Model(inputs=resnet.input, outputs=prediction)

In [51]:
resnet_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_______________________________________________________________________________________

In [52]:
resnet_model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [53]:
image_train.shape

(6141, 224, 224, 3)

In [54]:
label_train.shape

(6141, 102)

In [55]:
print(label_test[0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [56]:
image_test.shape

(820, 224, 224, 3)

In [57]:
resnet_model.fit(image_train, label_train, epochs = 30, batch_size = 32, verbose=1)

Epoch 1/30
192/192 [==============================] - 7s 35ms/step - loss: nan - accuracy: 0.0046
Epoch 2/30
192/192 [==============================] - 7s 34ms/step - loss: nan - accuracy: 0.0046
Epoch 3/30
192/192 [==============================] - 7s 34ms/step - loss: nan - accuracy: 0.0046
Epoch 4/30
192/192 [==============================] - 7s 34ms/step - loss: nan - accuracy: 0.0046
Epoch 5/30
192/192 [==============================] - 7s 34ms/step - loss: nan - accuracy: 0.0046
Epoch 6/30
192/192 [==============================] - 7s 34ms/step - loss: nan - accuracy: 0.0046
Epoch 7/30
192/192 [==============================] - 7s 35ms/step - loss: nan - accuracy: 0.0046
Epoch 8/30
192/192 [==============================] - 7s 35ms/step - loss: nan - accuracy: 0.0046
Epoch 9/30
192/192 [==============================] - 7s 34ms/step - loss: nan - accuracy: 0.0046
Epoch 10/30
192/192 [==============================] - 7s 34ms/step - loss: nan - accuracy: 0.0046
Epoch 11/30
192/192

# Evaluation

In [ ]:
resnet_model.save("resnet_pretrained_model.h5")

In [ ]:
preds = resnet_model.predict(image_test)

In [ ]:
preds

In [ ]:
label_test.shape

In [ ]:
from sklearn.metrics import accuracy_score
train_acc = accuracy_score(label_test, preds)

In [ ]:
resnet_model.evaluate(image_test, label_test)

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [ ]:
mlflow.keras.autolog()



with mlflow.start_run():
        
        #model.compile(loss=tf.keras.losses.categorical_crossentropy,
        #optimizer=tf.keras.optimizers.Adam(),
        #metrics=['accuracy'])
        # Fit our model
        history = model.fit(image_train, label_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(image_val, label_val))

        score = model.evaluate(image_test, label_test, batch_size=batch_size, verbose = 0)
        
        mlflow.log_param("activation function", act)
        mlflow.log_metric("test loss", score[0])
        mlflow.log_metric("test accuracy", score[1])
        mlflow.log_metric("test accuracy", score[1])
        
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        mlflow.keras.log_model(model, "standardCNN")        

In [ ]:
!mlflow ui

In [ ]:
#function that returns loss value & metrics values
model.evaluate(image_test, label_test)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=logs

In [ ]:
label_test.shape

In [ ]:
label_test[0][52]

In [ ]:
def prediction_accuracy(image_test, model):
    #going through each of the test images
    count = 0
    for i in range(len(image_test)): #going through all test images
        img = image_test[i] #load in the image
        
        img = img.reshape(-1, 100, 100, 3) #get it in the right shape for model.predict
        
        prediction = model.predict(img).flatten() #using model predict to get what it thinks is the answer
        
        # prediction.astype(np.int) #converts predictions to integers
        
        pred_index = np.argmax(prediction) # grabs the INDEX of the best prediction
        
        max_pred = max(prediction) # grabs the SCORE of best prediction
        # print(prediction)
        # print(pred_index)
        
        test_label_value = label_test[i][pred_index] # gets the actual test label at the same index
        # print(test_label_value)
        if test_label_value == 1:
            #print("CORRECT PREDICTION")
            count += 1
            
#         # FIND THE INDEX WHERE THE VALUE = 1
#         for j in range(len(predicted_label)):
#             if predicted_label[j] == 1 : #find the point when it finds 1 
#                 if label_test[i][j] == 1: #check if at this exact point is also 1  whic means correct prediction
#                     count+=1 #increase number of correct count by 1
#                     print(count)
#                     break
    print(f"num correct - {count}")
    accuracy = (count/len(label_test))*100
    per_symbol = '%'
    print(f'accuracy is {accuracy} {per_symbol}')        

In [ ]:
prediction_accuracy(image_test, model) 

In [ ]:
#checks for best model
runs = mlflow.search_runs(experiment_ids=experiment_id,
                          order_by=['metrics.mae'], max_results=1)

In [ ]:
runs.loc[0]

# Experiment InceptionV3

In [ ]:
# load model without output layer
from tensorflow.keras.applications.resnet50 import ResNet50
import keras
from tensorflow.keras import layers

In [ ]:
model = keras.applications.InceptionV3(include_top = True)

In [ ]:
print(model.summary())

In [ ]:
base_inputs = model.layers[0].input
base_outputs = model.layers[-2].output
final_outputs = layers.Dense(102)(base_outputs)
new_model = keras.Model(inputs=base_inputs, outputs=final_outputs)


In [ ]:
new_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"],
    )

In [ ]:
print(new_model.summary())

In [ ]:
new_model.fit(image_train, label_train, batch_size=80, epochs=15, verbose=2)

In [ ]:
preds = new_model.predict(image_test)

In [ ]:
new_model.evaluate(image_test, label_test)

# VGG 19 